## Evaluating the Models

In order to evaluate the performance of any model we need a distance metric. In this notebook we work through some commonly used distance metrics for evaluating learning to rank models. 
This notebook covers: 

1. mean squared error (MSE)
2. pairwise difference (PD)
3. Discounted Cumulative Gain (DCG)
4. Normalised Discounted Cumulative Gain (nDCG)

We start by loading in the data and fitting a basic linear regression model which we then use to make predictions for the test set. 

In [1]:
### We go ahead and load in the data, transform it, and build a model. 
### We make predictions for the data in the test set.

import pandas as pd
import numpy as np
from learningtorank import process
from sklearn.linear_model import LinearRegression

train = pd.read_csv("data/train.txt", header=None, sep=" ")
test = pd.read_csv("data/test.txt", header=None, sep=" ")

train_df = process.df_transform(train)
test_df = process.df_transform(test)

X = train_df[train_df.columns[2:]]
y = train_df[0]
reg = LinearRegression().fit(X, y)

X_test = test_df[test_df.columns[2:]]
y_test = test_df[0]

preds = reg.predict(X_test)

#### Mean Squared Error

Mean squared error (MSE) is often used to comapare the difference between true numeric values and predicted values. 

In the case of the MSLR Web-10K data set, each query document pair has a true relevance $t$ score between 0 and four, and we can use a model to obtain a predicted relevance score $p$. As such, mean squared error can be computed as the mean of $(t-p)^2$, across document-query pairs in the testing set.

We use the mean squared error function from the sci-kit learn module:

In [2]:
from sklearn.metrics import mean_squared_error

mean_squared_error(preds, y_test)

0.6055521003389398

If we think about a user of a search engine, they don't ever see that predicted value. They are instead presented with an ordered set of documents relating to their query. As such, mean squared error, which only takes into account the numeric values, is perhaps not the best distance metric for learning to rank.

#### Pairwise difference

Instead of considering the predicted and true numeric values associated with the query document pairs, pairwise difference considers only the ordering of the documents. Pariwise difference counts the number of "swaps" of neighbouring documents that have to be made to go from the predicted ordering to the true optimal ordering. The orderings are determined by ranking according to the predicted and true numeric values. 

Pairwise difference is computed for each query individually. The mean pariwise difference across all queries can then be computed and used to compare models. 

To compute the pairwsie difference, we use the bubble sort algorithm to transform the documents from the predicted order to the true optimal order, and count the nubmer of swaps implemented in the algorithm. 

In order to determine the Pairwise Difference we work with each of the queries individually. 

Once the data is split by query we need to 

In [3]:
preds_df = pd.DataFrame({'qid': test_df[1], 'truth' :test_df[0], 'predicted' : preds })

In [4]:
preds_df.sample(10)

,qid,truth,predicted
135905,16588,1,0.945433
15213,1993,1,0.964558
199799,23323,0,0.869287
140923,17188,0,0.686114
69636,8938,0,0.144627
20160,2638,0,0.856185
105088,13078,0,1.065212
163610,19483,0,0.686787
184150,21718,0,0.623244
200571,23443,1,1.081383


In [5]:
groups = preds_df.groupby('qid')

In [6]:
print(groups.size().min())
print(groups.size().max())

1
514


This shows that, for atleast one query, there is only 1 document for which a prediction was made. For such queries the list will always be in optimal order. We can either remove the contribution of these queries to the metric, or we can just leave them since they have the same effect on the metric of any odel considered. For now we keep them. 

Let's work with a specific query to compute the pairwise difference 

In [7]:
qid_13 = preds_df[preds_df.qid =='13']

In [8]:
qid_13.head()

,qid,truth,predicted
0,13,2,0.769230
1,13,1,0.293494
2,13,3,0.352380
3,13,1,0.634864
4,13,0,0.561271


The first thing we want to do is sort the list by "predicted".

In [9]:
qid_13.sort_values(by = "predicted").head()

,qid,truth,predicted
20,13,0,0.140340
1,13,1,0.293494
129,13,0,0.312027
6,13,1,0.348857
2,13,3,0.352380


We now want to count how many pairwise swaps have to be done by adjacently ordered. To do this we can consider simply the list of true values, in the order sorted by prediction like above. 

In [10]:
truth_by_pred = qid_13.sort_values(by = "predicted").truth.tolist()

In [11]:
def bubble_sort(vals_list):
    swaps = 0
    n=len(vals_list)
    sorted = 0
    while sorted == 0:
        swaps_this_pass = 0
        for i in range(0, n-1):

            if vals_list[i]>vals_list[i+1]:
                vals_list[i], vals_list[i+1] = vals_list[i+1], vals_list[i]
                swaps_this_pass = swaps_this_pass + 1

        if swaps_this_pass==0:
            sorted=1
        swaps = swaps + swaps_this_pass        
        n = n-1 #ith pass of bubble sort puts the nth value in order.
            #so there is no need to consider this. 
    return(swaps)

In [12]:
bubble_sort(truth_by_pred)

2641

Now we want to write a function which computes this number of pairwise swaps for each for each of the queries. 

In [13]:
def bubble_swaps(group):
    truth_by_pred = group.sort_values(by = "predicted").truth.tolist()
    swaps = bubble_sort(truth_by_pred)
    return swaps

In [14]:
pairwise = groups.apply(bubble_swaps)

In [15]:
pairwise.sample(10)

qid
13633       0
13543     571
13618    1078
7993     2184
9718     2633
14743       5
7573     1056
29968     199
23728     177
17173     972
dtype: int64

In [16]:
pairwise.mean()

2130.554

There are two aspects of ranking which are not caputed by the pairwise difference metric: 
1. we care about ordering at the start of the document list more than at the end

2. we care about ordering of documents less when the true values are similar.  


Pairwise difference weights the contribution of every "swap" equally in the distance metric: An out of order pair of documents the end of the list contribute the same amount of weight to the distance metric as an out of order pair at the start. 


However, if we think about a user of a search engine, they care much more about the order of the list of documents on the first page of search results than they do about the order on the last page. 

Discounted Cumulative Gain, or dcg, downweights the contributions of documents to the distance metric as you move down the list logarithmically, thereby accounting for the fact that we care more about the start of the list than the end. 


If we consider the following two lists of three documents. Their true values are: 
{10,7,9} and {10,8,9}, their pairwise difference from the truth is 1 for both lists, since we must swap the second and third document to get the either list to optimal order. 
However, ideally we would want to penalise 10,7,9 more than 10,8,9. 

The metric that fits all our needs is discounted cumulative gain, or dcg. 

In [17]:
import math

def dcg(ordered_data):
    """discounted cumulative gain"""
    n = len(ordered_data)
    if sum(ordered_data)==0:
        return 0
    else:
        indexloop = range(0, n)
        DCG = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
            DCG = DCG + current_ratio
        return(DCG)

In [18]:
dcg(truth_by_pred) #compute for one query

30.261924410467387

In [19]:
def sort_dcg(unordered_group):
    ordered_group = unordered_group.sort_values(by = "predicted").truth.tolist()
    return dcg(ordered_group)

In [20]:
dcg_score = groups.apply(sort_dcg)

In [21]:
dcg_score.sample(10)

qid
29278    20.979624
29803    33.537069
27448     0.000000
11218     1.896389
3208     12.793771
16378    18.896233
3598     14.914728
2068     14.234168
4948     35.064786
193      33.968572
dtype: float64

Although this distance metric has a couple of features we are happy with, it's hard to compare the numerical values across queries, since they all have different numbers of documents in them. 

Normalised discounted cumulative gain, or ndcg, 

In [22]:
""" this returns 0 if all of the ordered data is undesirable"""
def ndcg_p(ordered_data, p):
    """normalised discounted cumulative gain"""
    if sum(ordered_data)==0:
        return 0
    else:
        indexloop = range(0, p)
        DCG_p = 0
        for index in indexloop:
            current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
            DCG_p = DCG_p + current_ratio
        sorted_data= sorted(ordered_data,reverse=True)
        n = len(ordered_data)
        indexloop = range(0, n)
        iDCG_p = 0
        for index in indexloop:
            current_ratio=(2**(sorted_data[index])-1)*((math.log((index+2), 2))**(-1))
            iDCG_p = iDCG_p + current_ratio
        return(DCG_p/iDCG_p)

In [23]:
truth_by_pred = qid_13.sort_values(by = "predicted").truth.tolist()

In [24]:
try_ndgc=ndcg_p(truth_by_pred, 120)

In [25]:
try_ndgc

0.5495521971812585

In [26]:
truth_by_pred

[0,
 1,
 0,
 1,
 3,
 1,
 0,
 1,
 0,
 3,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 3,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 3,
 1,
 1,
 2,
 2,
 0,
 0,
 1,
 3,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 3,
 1,
 1,
 2,
 2,
 1,
 2,
 0]

In [27]:
def compute_at_p(ordered_data, p): 
    """
    returns -1 if number of items in list is less than p
    else returns ndcg@p
    """
    n = len(ordered_data)
    if n<p:
        result = np.nan
    else:
        result = ndcg_p(ordered_data, p)
    
    return result

In [28]:
def sort_ndcg(unordered_group, p):
    ordered_group = unordered_group.sort_values(by = "predicted").truth.tolist()
    return compute_at_p(ordered_group, p)

In [29]:
sort_ndcg(qid_13, 120)

0.5495521971812585

In [30]:
compute_at_p(truth_by_pred, 100)

0.4420443998743764

In [31]:
ndcg_score = groups.apply(sort_ndcg, p=100)

In [32]:
ndcg_score.sample(10)

qid
27838         NaN
3118          NaN
13573    0.441229
15748    0.428469
5833     0.419832
20938    0.298734
16183         NaN
29908    0.298554
25903    0.476342
13318    0.324499
dtype: float64

NaN represents that there was not enough documents considered for that query to compute the ndcg at the requested level, p. 

In order to summaries all these numbers into one we can use the `np.nanmean()` function, which ignores any `NaN` entries when computing the mean: 

In [33]:
np.nanmean(ndcg_score) 

0.34106516621584887